In [1]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.callbacks.manager import CallbackManager
from langchain.document_loaders import PyPDFLoader
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.embeddings import OpenAIEmbeddings

# from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings


USER_AGENT environment variable not set, consider setting it to identify your requests.


# llama3 실험

In [2]:
# LangChain이 지원하는 다른 채팅 모델을 사용합니다. 여기서는 Ollama를 사용합니다.
llm = ChatOllama(
    model="Bllossom-8B-Q4_K_M:latest",
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
)

/var/folders/jn/6v9ffl5x6gvf4y16kqq_wqgh0000gn/T/ipykernel_6290/1013373852.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(
/var/folders/jn/6v9ffl5x6gvf4y16kqq_wqgh0000gn/T/ipykernel_6290/1013373852.py:2: DeprecationWarning: callback_manager is deprecated. Please use callbacks instead.
  llm = ChatOllama(


In [3]:
# 주제를 기반으로 짧은 농담을 요청하는 프롬프트 템플릿을 생성합니다.
prompt = ChatPromptTemplate.from_template("{topic} 에 대하여 간략히 설명해 줘.")

In [4]:
# LangChain 표현식 언어 체인 구문을 사용합니다.
chain = prompt | llm | StrOutputParser()

In [5]:
# 간결성을 위해 응답은 터미널에 출력됩니다.
# 프로덕션 환경에서 애플리케이션을 배포하기 위해 LangServe를 사용할 수 있습니다.
_ = chain.invoke({"topic": "deep learning"})

물론입니다! deep learning은 인공지능의 한 분야로, 인간의 뇌를 모방하여 데이터로부터 학습하는 알고리즘을 개발합니다. 주로 신경망(neural network)을 사용하며, 이를 통해 이미지 인식, 음성 인식, 자연어 처리 등 다양한 작업을 수행할 수 있습니다. deep learning은 데이터가 충분히 많고 다양하다면 매우 정확한 예측과 분류를 가능하게 합니다.

In [6]:
_ = chain.invoke({"topic": "대한민국의 수도는?"})

안녕하세요! 대한민국의 수도는 서울입니다. 서울은 한반도 중앙부에 위치한 도시로, 역사와 문화가 풍부한 곳입니다. 1948년부터 현재까지 대한민국의 수도를 맡고 있으며, 많은 관광 명소와 전통 시장, 현대적인 건축물들이 공존하는 특징을 가지고 있습니다. 또한, 서울은 경제적, 정치적 중심지로서 대한민국의 주요 산업과 문화가 집중된 곳입니다.

# RAG

In [3]:
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [ ]:
loader = PyPDFLoader('/Users/baeahram/Desktop/git/RAG_DataDriven/cache/files/빅데이터분석을통한마케팅활용에관한연구.pdf')

docs = loader.load()

print(f'문서의 수: {len(docs)}')

ValueError: File path /Users/baeahram/Desktop/git/RAG_DataDriven/docs/빅데이터분석을통한마케팅활용에관한연구.pdf is not a valid file or url

In [16]:
print(docs)

[Document(metadata={'source': '/Users/baeahram/Desktop/git/RAG_DataDriven/docs/빅데이터분석을통한마케팅활용에관한연구.pdf', 'page': 0}, page_content=' \n \n저작자표시-비영리-변경금지 2.0 대한민국 \n이용자는 아래의 조건을 따르는 경우에 한하여 자유롭게 \nl 이 저작물을 복제, 배포, 전송, 전시, 공연 및 방송할 수 있습니다.  \n다음과 같은 조건을 따라야 합니다: \nl 귀하는, 이 저작물의 재이용이나 배포의 경우, 이 저작물에 적용된 이용허락조건\n을 명확하게 나타내어야 합니다.  \nl 저작권자로부터 별도의 허가를 받으면 이러한 조건들은 적용되지 않습니다.  \n저작권법에 따른 이용자의 권리는 위의 내용에 의하여 영향을 받지 않습니다. \n이것은 이용허락규약(Legal Code)을 이해하기 쉽게 요약한 것입니다.  \nDisclaimer  \n  \n  \n저작자표시. 귀하는 원저작자를 표시하여야 합니다. \n비영리. 귀하는 이 저작물을 영리 목적으로 이용할 수 없습니다. \n변경금지. 귀하는 이 저작물을 개작, 변형 또는 가공할 수 없습니다. \n'), Document(metadata={'source': '/Users/baeahram/Desktop/git/RAG_DataDriven/docs/빅데이터분석을통한마케팅활용에관한연구.pdf', 'page': 1}, page_content='공학석사학위논문\n빅데이터분석을통한마케팅활용에관한연구-온라인마케팅을중심으로-\n아주대학교IT융합대학원IT융합공학과윤태구\n'), Document(metadata={'source': '/Users/baeahram/Desktop/git/RAG_DataDriven/docs/빅데이터분석을통한마케팅활용에관한연구.pdf', 'page': 2}, page_content='빅데이터분석을통한마케팅활용에관한연구-온라인마케팅을중심으로-\nAStudyontheMarketingUtilizationthro

In [17]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, chunk_overlap = 50
)

splits = text_splitter.split_documents(docs)

In [18]:
len(splits)

83

In [19]:
verctorstore = FAISS.from_documents(
    documents=splits, embedding=OpenAIEmbeddings()
)

retriever = verctorstore.as_retriever()

/var/folders/jn/6v9ffl5x6gvf4y16kqq_wqgh0000gn/T/ipykernel_98580/3230673571.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  documents=splits, embedding=OpenAIEmbeddings()


In [20]:
template = (    
    "당신은 질문에 한글로 답변하는 업무를 수행하는 도우미입니다. 제공된 문맥 정보를 활용하여 질문에 답변하세요." 
    "답을 모를 경우, 모른다고 말하세요. 최대 세 문장으로 답변을 작성하고, 간결하게 표현하세요."
    
    "Question: {question}\n"
    "Context: {context}\n"
    "Answer:"
)

prompt = ChatPromptTemplate.from_template(template)


In [21]:
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

rag_chain = (
    {'context': retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke(
    '빅 데이터의 세계 시장 규모에 대해 설명해주세요.'
)

안녕하세요! 빅데이터의 세계 시장 규모에 대해 설명해드리겠습니다. 2012년 기준으로는 약 51억 달러였던 빅데이터 시장이, 2017년에는 534억 달러로 크게 성장했습니다. 국내 빅데이터 시장도 2013년 1억 6,300만 달러에서 2017년 4억 2,300만 달러로, 2020년에는 약 8억 9,400만 달러로 성장하고 있습니다. 앞으로도 시장이 지속해서 성장할 것으로 예상됩니다. 구글과 유튜브 같은 선진 기업들은 무료 혹은 저가의 서비스를 제공하면서 이용자의 데이터를 수집

In [ ]:
openai_embedding = OpenAIEmbeddings(model="text-embedding-3-small", dimensions = 1024)

store = LocalFileStore('cache/')

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    openai_embedding, store, namespace = openai_embedding.model
)